In [0]:
# No notebook crie código Spark que respondam as seguintes perguntas: 
# - Qual país possui a maior quantidade de itens cancelados? 
# - Qual o faturamento da linha de produto mais vendido, considere como os itens Shipped, cujo o pedido foi realizado no ano de 2005? 
# - Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail deve estar mascarado.

# para este case estudo foi realizado a criação da tabela no banco de dados "" feito pela Case 4,  deste modo com todas as tabelas ja criadas e populadas para este caso vou utilizar a tabela para as analises abaixo:


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [0]:
#subindo para um dataframe as tabela Necessarias para analise:

customers_df = spark.table('ecommerce_rnp_teste.tcustomers')

orders_df = spark.table('ecommerce_rnp_teste.torders')

orderdetails_df = spark.table('ecommerce_rnp_teste.torderdetails')

employees_df = spark.table('ecommerce_rnp_teste.temployees')

offices_df = spark.table('ecommerce_rnp_teste.toffices')

payments_df = spark.table('ecommerce_rnp_teste.tpayments')

productlines_df = spark.table('ecommerce_rnp_teste.tproductlines')

products_df = spark.table('ecommerce_rnp_teste.tproducts')


In [0]:
#Query join em Spark:
result = (
    customers_df.alias("c")
    .join(orders_df.alias("o"), F.col("c.customer_number") == F.col("o.customer_number"))
    .join(orderdetails_df.alias("od"), F.col("o.order_number") == F.col("od.order_number"))
    .filter(F.col("o.status") == "Cancelled")
    .groupBy("c.country")
    .agg(F.sum("od.quantity_ordered").alias("CancelledItems"))
    .orderBy(F.desc("CancelledItems"))
)

display(result)
# Salvar o resultado no formato Delta
result.write.format("delta").mode("overwrite").save("/FileStore/output/delta_analise1")

#verificando o arquivo delta
analise1 = spark.read.format("delta").load("/FileStore/output/delta_analise1")
display(analise1)

country,CancelledItems
Spain,605
New Zealand,596
Sweden,550
USA,454
UK,429


Databricks visualization. Run in Databricks to view.

country,CancelledItems
Spain,605
New Zealand,596
Sweden,550
USA,454
UK,429


In [0]:
# ANALISE 2
# Qual o faturamento da linha de produto mais vendido, considere como os itens Shipped, cujo o pedido foi realizado no ano de 2005?

#Query join em Spark:
result2 = (
    orders_df.alias("o")
    .join(orderdetails_df.alias("od"), F.col("o.order_number") == F.col("od.order_number"))
    .join(products_df.alias("p"), F.col("p.product_code") == F.col("od.product_code"))
    .filter((F.col("o.status") == "Shipped") & (F.year("o.order_date") == 2005))
    .groupBy("p.product_line")
    .agg(F.sum(F.col("od.quantity_ordered") * F.col("od.price_each")).alias("faturamento_total"))
    .orderBy(F.desc("faturamento_total"))
)

display(result2)

# Salvar o resultado no formato Delta
#result2.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("/FileStore/output/delta_analise2") - somente desmarque se preicsar fazer um merge dos files
result2.write.format("delta").mode("overwrite").save("/FileStore/output/delta_analise2")

#verificando o arquivo delta
analise2 = spark.read.format("delta").load("/FileStore/output/delta_analise2")
display(analise2)

product_line,faturamento_total
Classic Cars,603666.9900000006
Vintage Cars,222510.69999999995
Motorcycles,212684.55000000002
Trucks and Buses,182231.44999999998
Planes,109701.55999999997
Ships,62989.19
Trains,22311.260000000002


Databricks visualization. Run in Databricks to view.

product_line,faturamento_total
Classic Cars,603666.9900000006
Vintage Cars,222510.69999999995
Motorcycles,212684.55000000002
Trucks and Buses,182231.44999999998
Planes,109701.55999999997
Ships,62989.19
Trains,22311.260000000002


In [0]:

#Query join em Spark:
result3 = (
    employees_df.alias("e")
    .join(offices_df.alias("o"), F.col("e.office_code") == F.col("o.office_code"))
    .filter((F.col("o.country") == "Japan") & (F.col("e.job_Title").like("%Sales%")))
    .select(
        F.col("e.first_name").alias("Nome"),
        F.col("e.last_name").alias("Sobrenome"),
        F.concat(
            F.lit('*****'),
            F.expr("substring(e.email, position('@' in e.email), length(e.email))")
        ).alias("Email_Mascarado")
    )
)

display(result3)

# Salvar o resultado no formato Delta
result3.write.format("delta").mode("overwrite").save("/FileStore/output/delta_analise3")

#verificando o arquivo delta
analise3 = spark.read.format("delta").load("/FileStore/output/delta_analise3")
display(analise3)


Nome,Sobrenome,Email_Mascarado
Mami,Nishi,*****@classicmodelcars.com
Yoshimi,Kato,*****@classicmodelcars.com


Nome,Sobrenome,Email_Mascarado
Mami,Nishi,*****@classicmodelcars.com
Yoshimi,Kato,*****@classicmodelcars.com


In [0]:
#Validading Files Delta geral
display(dbutils.fs.ls("dbfs:/FileStore/output"))
display(dbutils.fs.ls("dbfs:/FileStore/output/delta_analise1/"))
display(dbutils.fs.ls("dbfs:/FileStore/output/delta_analise2/"))
display(dbutils.fs.ls("dbfs:/FileStore/output/delta_analise3/"))

path,name,size,modificationTime
dbfs:/FileStore/output/delta_analise1/,delta_analise1/,0,0
dbfs:/FileStore/output/delta_analise2/,delta_analise2/,0,0
dbfs:/FileStore/output/delta_analise3/,delta_analise3/,0,0


path,name,size,modificationTime
dbfs:/FileStore/output/delta_analise1/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/output/delta_analise1/part-00000-b2c6e6e1-5b1a-4fb1-8e5d-f307c3ab9be5-c000.snappy.parquet,part-00000-b2c6e6e1-5b1a-4fb1-8e5d-f307c3ab9be5-c000.snappy.parquet,947,1692909296000


path,name,size,modificationTime
dbfs:/FileStore/output/delta_analise2/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/output/delta_analise2/part-00000-c1cbf167-2424-4f1c-98d3-bda00942b2b2-c000.snappy.parquet,part-00000-c1cbf167-2424-4f1c-98d3-bda00942b2b2-c000.snappy.parquet,1072,1692909321000


path,name,size,modificationTime
dbfs:/FileStore/output/delta_analise3/_delta_log/,_delta_log/,0,0
dbfs:/FileStore/output/delta_analise3/part-00000-2651bb2f-5925-4104-977f-fec7bd956ec0-c000.snappy.parquet,part-00000-2651bb2f-5925-4104-977f-fec7bd956ec0-c000.snappy.parquet,1298,1692909584000
